In [1]:
import pandas as pd
import numpy as np
import h5py
import os
from IPython.display import display
from nltk.tokenize import sent_tokenize
import nltk.data
from collections import Counter
import pickle
from utils.create_datasets import Entry
from utils.create_datasets import process_entry

/home/chenxiang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chenxiang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data_dir = '../data/'
test_data = 'test_data.csv'
test_file = os.path.join(data_dir, test_data)
os.path.exists(test_file)

True

In [3]:
test_data = pd.read_csv(test_file, encoding='utf-8', index_col=None)
display(test_data.head())

,id,content,title
0,1,This was the year that felt like 50 years. We ...,21 Stories Our Readers Loved in 2017
1,2,Gary Vaynerchuk once told a 20 year old Taylor...,What To Do After Graduating College
2,3,"On this episode of Recode Decode, hosted by Ka...",Full transcript: Former Groupon CEO Andrew Mas...
3,4,Today we are going to talk about the Albigensi...,The Albigensian Crusade And The Black Mass
4,5,IntroductionThe more you want to impact the re...,The Surprising Secret To Changing The World


In [4]:
with open(os.path.join(data_dir, 'Vocab.pkl'), 'rb') as f:
    Vocab_ = pickle.load(f)
print(Vocab_['word2id'])

{'PAD': 0, 'EOS': 50001, 'SOS': 50002, 'UNK': 50003, 'w+': 1}


In [6]:
content_example, title_example = test_data.content[0], test_data.title[0]
entry_example = Entry(content_example, title_example, Vocab_)

In [7]:
from utils.create_datasets import process_entry
encoder_input, decoder_input, target = process_entry(entry_example)

In [8]:
display(encoder_input)
display(decoder_input)
display(target)

[50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,


[50002,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

[50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50003,
 50001,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]